In [ ]:
import numpy as np
import pandas as pd
import requests


match_id_list = [117404]   # add more match IDs here

# Your API details
headers = {
    "x-rapidapi-key": "d1f52dc134msh606acc9612e6217p15c440jsnb9910abbb802",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

def flatten_column(df, colname, prefix):
    if colname in df.columns:
        nested = pd.json_normalize(df[colname].iloc[0])
        nested = nested.add_prefix(prefix)
        df = df.drop(columns=[colname]).join(nested)
    return df

def create_players_df(item):
    df = pd.DataFrame(item["player"])
    df["category"] = item["category"]
    return df



def fetch_team_squad(match_id, team_id, series_id, headers):
    squad_url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/team/{team_id}"
    response = requests.get(squad_url, headers=headers)
    team_data = response.json()

    # team_data structure: { ..., "something": [ { "player": [...], "category": "Playing XI" }, ... ] }
    for key, value in team_data.items():
        if not isinstance(value, list):
            continue

        for item in value:
            category_raw = item.get("category", "")
            cat_key = category_raw.lower().replace(" ", "_")  # "Playing XI" -> "playing_xi"

            if cat_key not in rows_by_category:
                # in case new categories appear
                rows_by_category[cat_key] = []

            for p in item.get("player", []):
                row = dict(p)  # copy player dict
                row["category"] = category_raw
                row["match_id"] = match_id
                row["series_id"] = series_id
                row["team_id"] = team_id
                rows_by_category[cat_key].append(row)




match_header = []
playing_11 = []
bench_players = []
supports_staff = []


rows_by_category = {
    "playing_xi": [],
    "bench": [],
    "support_staff": []
}




dfs = {}

for match_id in match_id_list:
    try:
        match_header_url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}"
        response = requests.get(match_header_url, headers=headers)
        data = response.json()

        df = pd.DataFrame([data])

        # Flatten nested JSON fields
        nested_fields = {
            "team1": "team1_",
            "team2": "team2_",
            "umpire1": "umpire1_",
            "umpire2": "umpire2_",
            "umpire3": "umpire3_",
            "referee": "referee_",
            "venueinfo": "venueinfo_",
        }

        for col, prefix in nested_fields.items():
            df = flatten_column(df, col, prefix)

        # Drop columns you don't need
        df = df.drop(columns=["appindex", "boundarytrackervalues"], errors='ignore')

        match_header.append(df)
    except Exception as e:
        print(f"Error processing match ID {match_id}: {e}")


    # temp_df = match_header[0]
    # team1id = temp_df[['team1_teamid']].iloc[0].tolist()
    # team2id = temp_df[['team2_teamid']].iloc[0].tolist()
    # series_id = temp_df['seriesid']



    # squad_url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/team/{team1id[0]}"
    # response = requests.get(squad_url, headers=headers)
    # team1_data = response.json()

    # for key, value in team1_data.items():
    #     for item in value:
    #         cat = item["category"].lower().replace(" ", "_")   # e.g. "support staff" → "support_staff"
    #         dfs[cat] = pd.DataFrame(item["player"])
    #         dfs[cat]["category"] = item["category"]

    



    temp_row = match_header[0]
    team1_id = temp_row["team1_teamid"]
    team2_id = temp_row["team2_teamid"]
    series_id = temp_row["seriesid"]


    # --- fetch for both teams ---
    for tid in [team1_id, team2_id]:
        fetch_team_squad(match_id, tid, series_id, headers)

    


for cat, rows in rows_by_category.items():
    if rows:  # only create if there is data
        dfs[cat] = pd.DataFrame(rows)




# for name, df in dfs.items():
#     df.to_csv(f"{name}.csv", index=False)


        



# final_df = pd.concat(match_header, ignore_index=True)

# print(final_df.head())
# print(final_df.columns)

print(dfs)



{}


In [ ]:
import numpy as np
import pandas as pd
import requests


match_id_list = [117404]   # add more match IDs here

# Your API details
headers = {
    "x-rapidapi-key": "d1f52dc134msh606acc9612e6217p15c440jsnb9910abbb802",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

def flatten_column(df, colname, prefix):
    if colname in df.columns:
        nested = pd.json_normalize(df[colname].iloc[0])
        nested = nested.add_prefix(prefix)
        df = df.drop(columns=[colname]).join(nested)
    return df

def create_players_df(item):
    df = pd.DataFrame(item["player"])
    df["category"] = item["category"]
    return df



def fetch_team_squad(match_id, team_id, series_id, headers):
    squad_url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/team/{team_id}"
    response = requests.get(squad_url, headers=headers)
    team_data = response.json()

    # team_data structure: { ..., "something": [ { "player": [...], "category": "Playing XI" }, ... ] }
    for key, value in team_data.items():
        if not isinstance(value, list):
            continue

        for item in value:
            category_raw = item.get("category", "")
            cat_key = category_raw.lower().replace(" ", "_")  # "Playing XI" -> "playing_xi"

            if cat_key not in rows_by_category:
                # in case new categories appear
                rows_by_category[cat_key] = []

            for p in item.get("player", []):
                row = dict(p)  # copy player dict
                row["category"] = category_raw
                row["match_id"] = match_id
                row["series_id"] = series_id
                row["team_id"] = team_id
                rows_by_category[cat_key].append(row)




match_header = []

rows_by_category = {
    "playing_xi": [],
    "bench": [],
    "support_staff": []
}


for match_id in match_id_list:
    try:
        match_header_url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}"
        response = requests.get(match_header_url, headers=headers)
        data = response.json()

        df = pd.DataFrame([data])

        # Flatten nested JSON fields
        nested_fields = {
            "team1": "team1_",
            "team2": "team2_",
            "umpire1": "umpire1_",
            "umpire2": "umpire2_",
            "umpire3": "umpire3_",
            "referee": "referee_",
            "venueinfo": "venueinfo_",
        }

        for col, prefix in nested_fields.items():
            df = flatten_column(df, col, prefix)

        # Drop columns you don't need
        df = df.drop(columns=["appindex", "boundarytrackervalues"], errors='ignore')

        match_header.append(df)
    except Exception as e:
        print(f"Error processing match ID {match_id}: {e}")


    temp_row = match_header[0]
    team1_id = temp_row["team1_teamid"]
    team2_id = temp_row["team2_teamid"]
    series_id = temp_row["seriesid"]


    # --- fetch for both teams ---
    for tid in [team1_id, team2_id]:
        fetch_team_squad(match_id, tid, series_id, headers)

    


for cat, rows in rows_by_category.items():
    if rows:  # only create if there is data
        dfs[cat] = pd.DataFrame(rows)




{'playing_xi': [], 'bench': [], 'support_staff': []}


In [19]:
import numpy as np
import pandas as pd
import requests

match_id_list = [117404]   # add more match IDs here

headers = {
    "x-rapidapi-key": "d1f52dc134msh606acc9612e6217p15c440jsnb9910abbb802",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

def flatten_column(df, colname, prefix):
    if colname in df.columns and pd.notna(df[colname].iloc[0]):
        nested = pd.json_normalize(df[colname].iloc[0])
        nested = nested.add_prefix(prefix)
        df = df.drop(columns=[colname]).join(nested)
    return df

# ---------- players store ----------
rows_by_category = {
    "playing_xi": [],
    "bench": [],
    "support_staff": []
}

def fetch_team_squad(match_id, team_id, series_id):
    squad_url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/team/{team_id}"
    response = requests.get(squad_url, headers=headers)
    team_data = response.json()

    for key, value in team_data.items():
        if not isinstance(value, list):
            continue

        for item in value:
            category_raw = item.get("category", "")
            cat_key = category_raw.lower().replace(" ", "_")  # Playing XI -> playing_xi

            if not cat_key:
                continue

            if cat_key not in rows_by_category:
                rows_by_category[cat_key] = []

            for p in item.get("player", []):
                row = dict(p)
                row["category"] = category_raw
                row["match_id"] = match_id
                row["series_id"] = series_id
                row["team_id"] = team_id
                rows_by_category[cat_key].append(row)

# ---------- match headers store ----------
match_header_dfs = []   # list of per-match header DataFrames

# =============== MAIN LOOP ===============
for match_id in match_id_list:
    try:
        # 1) Fetch match header
        match_header_url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}"
        response = requests.get(match_header_url, headers=headers)
        data = response.json()

        df = pd.DataFrame([data])

        # Flatten nested JSON fields
        nested_fields = {
            "team1": "team1_",
            "team2": "team2_",
            "umpire1": "umpire1_",
            "umpire2": "umpire2_",
            "umpire3": "umpire3_",
            "referee": "referee_",
            "venueinfo": "venueinfo_",
        }

        for col, prefix in nested_fields.items():
            df = flatten_column(df, col, prefix)

        df = df.drop(columns=["appindex", "boundarytrackervalues"], errors='ignore')

        # add explicit match_id column (sometimes it's only inside JSON)
        df["match_id"] = match_id

        # keep this for final match_header_df
        match_header_dfs.append(df)

        # 2) Extract IDs for players API
        row = df.iloc[0]
        team1_id = row["team1_teamid"]
        team2_id = row["team2_teamid"]
        series_id = row["seriesid"]

        # 3) Fetch squads for both teams
        for tid in [team1_id, team2_id]:
            fetch_team_squad(match_id, tid, series_id)

    except Exception as e:
        print(f"Error processing match ID {match_id}: {e}")

# =============== BUILD FINAL DFs ===============

# 🔹 match header (one row per match)
match_header_df = pd.concat(match_header_dfs, ignore_index=True)

# 🔹 team-level header (one row per team per match)
team_header_rows = []
for _, r in match_header_df.iterrows():
    for side in ["team1", "team2"]:
        team_header_rows.append({
            "match_id": r["match_id"],
            "series_id": r["seriesid"],
            "team_side": side,
            "team_id": r[f"{side}_teamid"],
            "team_name": r.get(f"{side}_teamname", None),
            "team_sname": r.get(f"{side}_teamsname", None),
            "team_flag_imageid": r.get(f"{side}_imageid", None),
            "venue": r.get("venueinfo_ground", None),
            "city": r.get("venueinfo_city", None)
        })

team_header_df = pd.DataFrame(team_header_rows)

# 🔹 players by category
dfs = {}
for cat, rows in rows_by_category.items():
    if rows:
        dfs[cat] = pd.DataFrame(rows)

# =============== CHECK OUTPUTS ===============
print("match_header_df columns:", match_header_df.columns.tolist())
print("\nteam_header_df head():")
print(team_header_df.head())

print("\nAvailable player dfs:", list(dfs.keys()))
print("\nPlaying XI sample:")
print(dfs["playing_xi"].head())


match_header_df columns: ['matchid', 'seriesid', 'seriesname', 'matchdesc', 'matchformat', 'startdate', 'enddate', 'state', 'status', 'currbatteamid', 'ismatchnotcovered', 'istournament', 'tossstatus', 'seriesstartdt', 'seriesenddt', 'istimeannounced', 'statetitle', 'replacematchid', 'livestreamenabled', 'isfantasyenabled', 'broadcastinfo', 'daynight', 'matchtype', 'seriestype', 'shortstatus', 'matchimageid', 'islivestreamenabled', 'isfantasyhandbookenabled', 'team1_teamid', 'team1_teamname', 'team1_teamsname', 'team1_isfullmember', 'team1_isassociated', 'team1_isleagueteam', 'team1_iswomenteam', 'team1_isheader', 'team1_isactive', 'team1_teampriority', 'team1_isvideopresent', 'team1_imageid', 'team1_countryname', 'team1_belongsto', 'team2_teamid', 'team2_teamname', 'team2_teamsname', 'team2_isfullmember', 'team2_isassociated', 'team2_isleagueteam', 'team2_iswomenteam', 'team2_isheader', 'team2_isactive', 'team2_teampriority', 'team2_isvideopresent', 'team2_imageid', 'team2_countryname

In [27]:
import pandas as pd
import requests

match_id_list = [117404, 126669, 108793]   # add more match IDs here

headers = {
    "x-rapidapi-key": "d1f52dc134msh606acc9612e6217p15c440jsnb9910abbb802",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

def flatten_column(df, colname, prefix):
    if colname in df.columns and pd.notna(df[colname].iloc[0]):
        nested = pd.json_normalize(df[colname].iloc[0])
        nested = nested.add_prefix(prefix)
        df = df.drop(columns=[colname]).join(nested)
    return df

# store ALL players from ALL matches + BOTH teams
rows_by_category = {
    "playing_xi": [],
    "bench": [],
    "support_staff": []
}

def fetch_team_squad(match_id, team_id, series_id):
    squad_url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/team/{team_id}"
    response = requests.get(squad_url, headers=headers)
    team_data = response.json()

    for key, value in team_data.items():
        if not isinstance(value, list):
            continue

        for item in value:
            category_raw = item.get("category", "")
            cat_key = category_raw.lower().replace(" ", "_")  # "Playing XI" -> "playing_xi"

            if cat_key not in rows_by_category:
                rows_by_category[cat_key] = []

            for p in item.get("player", []):
                row = dict(p)
                row["category"] = category_raw
                row["match_id"] = match_id
                row["series_id"] = series_id
                row["team_id"] = team_id
                rows_by_category[cat_key].append(row)


match_header_dfs = []

for match_id in match_id_list:
    match_header_url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}"
    response = requests.get(match_header_url, headers=headers)
    data = response.json()

    df = pd.DataFrame([data])

    nested_fields = {
        "team1": "team1_",
        "team2": "team2_",
        "umpire1": "umpire1_",
        "umpire2": "umpire2_",
        "umpire3": "umpire3_",
        "referee": "referee_",
        "venueinfo": "venueinfo_",
    }

    for col, prefix in nested_fields.items():
        df = flatten_column(df, col, prefix)

    df = df.drop(columns=["appindex", "boundarytrackervalues"], errors='ignore')
    df["match_id"] = match_id
    match_header_dfs.append(df)

    # get scalar IDs
    row = df.iloc[0]
    team1_id = row["team1_teamid"]
    team2_id = row["team2_teamid"]
    series_id = row["seriesid"]

    # fetch squads for BOTH teams → all go into rows_by_category
    for tid in [team1_id, team2_id]:
        fetch_team_squad(match_id, tid, series_id)

# build final category-level DataFrames (both teams combined)
dfs = {}
for cat, rows in rows_by_category.items():
    if rows:
        dfs[cat] = pd.DataFrame(rows)

# convenient variables
df_playing_xi = dfs.get("playing_xi", pd.DataFrame())
df_bench = dfs.get("bench", pd.DataFrame())
df_support_staff = dfs.get("support_staff", pd.DataFrame())


# print(df_playing_xi)
# print("---------------------")
# print(df_bench)
# print("---------------------")
# print(df_support_staff)

match_header_df = pd.concat(match_header_dfs, ignore_index=True)
match_header_df.to_csv("match_header.csv", index=False)

# Export players by category
df_playing_xi.to_csv("playing_xi.csv", index=False)
df_bench.to_csv("bench.csv", index=False)
df_support_staff.to_csv("support_staff.csv", index=False)



In [29]:
import pandas as pd
import requests

match_id_list = [117404]   # add more match IDs here

headers = {
    "x-rapidapi-key": "d1f52dc134msh606acc9612e6217p15c440jsnb9910abbb802",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

def flatten_column(df, colname, prefix):
    if colname in df.columns and pd.notna(df[colname].iloc[0]):
        nested = pd.json_normalize(df[colname].iloc[0])
        nested = nested.add_prefix(prefix)
        df = df.drop(columns=[colname]).join(nested)
    return df


match_header_dfs = []

for match_id in match_id_list:
    match_header_url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}"
    response = requests.get(match_header_url, headers=headers)
    data = response.json()

    df = pd.DataFrame([data])

    nested_fields = {
        "team1": "team1_",
        "team2": "team2_",
        "umpire1": "umpire1_",
        "umpire2": "umpire2_",
        "umpire3": "umpire3_",
        "referee": "referee_",
        "venueinfo": "venueinfo_",
    }

    for col, prefix in nested_fields.items():
        df = flatten_column(df, col, prefix)

    df = df.drop(columns=["appindex", "boundarytrackervalues"], errors='ignore')
    df["match_id"] = match_id
    match_header_dfs.append(df)

    # get scalar IDs
    row = df.iloc[0]
    team1_id = row["team1_teamid"]
    team2_id = row["team2_teamid"]
    match_start_time = row["startdate"]
    match_end_time = row["enddate"]

    print(match_start_time,"--------", match_end_time)
    


    





1765008000000 -------- 1765033861499


In [67]:
final_df.to_csv("all_matches.csv", index=False)